# Resolver

In this notebook we will resolve the fully sidecar DID created in the [Alexis](./Alexis.ipynb). See step A2.4. It is recommended that you run through steps A through D, whereby DID controllers ([Alexis](./Alexis.ipynb) and [Satoshi](./Satoshi.ipynb)) join an sparse merkle tree aggregated beacon managed by an [Aggregator](./Aggregator.ipynb). They then include this beacon in an offline, sidecar DID document and generate the associated did:btc identifier. After, Alexis submits an update to the Aggregator who coordinates with Alexis and Satoshi to authorize the update before broadcasting it to the bitcoin network. 

The high level steps required for resolutionsare as follows:

1. Get sidecar history from DID controller
2. Verify initial DID document against DID
3. Initialize resolution variables
4. Collect beacon signals (btc txs) from bitcoin for all beacons
5. Process beacon signals to retrieve DID updates according to beacon type
6. Apply DID document transformations from updates in order

Note: The resolution algorithm is iterative. After applying an update from beacon signal to a DID document, resolution must check if there have been changes to the beacons in the DID document. If there have, the algorithm must return to step 4 and update the set of beacon signals. 

Resolution is complete only when all beacon signals from all beacons have been processed and their updates applied AND the updates construct a complete version history of the DID document up to its current state.

In [1]:
import sys
import os

# Get the path to the Notebooks directory
notebooks_path = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the Notebooks directory to the sys.path
sys.path.append(notebooks_path)

## R1. Get sidecar history from DID controller

The DID controller must provide:

Note: We should also include some meta data about how many updates there have been. Latest version?

- The initial DID document (if offline sidecar)
- The current DID document
- The proof data for each beacon signal
    - txid: bitcoin transaction identifier of beacon signal
    - merkle proof of (non)inclusion
    - DID update invocation (if update included in beacon signal)
 
Paste in the output from the end of the [Alexis](./Alexis.ipynb) notebook.

In [2]:
did_to_resolve = 'did:btc1:x1x03w9d88pwcj7cm4tjht5w3lg3rg2zjqzv072q2n2y8y2g9shqks5c7sl5'
sidecar_history = {'initial_document': {'@context': ['https://www.w3.org/ns/did/v1'], 'verificationMethod': [{'id': '#initialKey', 'type': 'JsonWebKey', 'publicKeyJwk': {'kty': 'EC', 'crv': 'secp256k1', 'x': '0233985507371e4a22cd2acc2cfed604efb2fc24400ff4a6bce8086561244f5a6d'}, 'controller': 'did:btc1:x1x03w9d88pwcj7cm4tjht5w3lg3rg2zjqzv072q2n2y8y2g9shqks5c7sl5'}], 'authentication': ['#initialKey'], 'assertionMethod': ['#initialKey'], 'capabilityInvocation': ['#initialKey'], 'capabilityDelegation': ['#initialKey'], 'service': [{'id': '#initial_p2pkh', 'type': 'SingletonBeacon', 'serviceEndpoint': 'bitcoin:mvrk1NC2TWi3gnSUkD9NMyRFqtBGFdXhHv'}, {'id': '#initial_p2wpkh', 'type': 'SingletonBeacon', 'serviceEndpoint': 'bitcoin:tb1q4prl9he7s99639g5kzp6pencna25ykmdd08h0k'}, {'id': '#initial_p2tr', 'type': 'SingletonBeacon', 'serviceEndpoint': 'bitcoin:tb1p9s5fa89ypc6dnqmxkmclxa3ecfqvrfqchejq7d89agccm9c7u5fs86akaw'}, {'id': '#smt_aggregated', 'type': 'SMTAggregatorBeacon', 'serviceEndpoint': 'bitcoin:tb1pfdnyc8vxeca2zpsg365sn308dmrpka4e0n9c5axmp2nptdf7j6ts7eqhr8'}], 'id': 'did:btc1:x1x03w9d88pwcj7cm4tjht5w3lg3rg2zjqzv072q2n2y8y2g9shqks5c7sl5'}, 'signals': {'bb1127988db99a6fa710cec1f7514de8995b69ccc6bfeb89a6877a5424ce69c8': {'update': {'@context': ['https://w3id.org/security/v2', 'https://w3id.org/zcap/v1', 'https://w3id.org/json-ld-patch/v1'], 'patch': [{'op': 'add', 'path': '/service/4', 'value': {'id': '#linked-domain', 'type': 'LinkedDomains', 'serviceEndpoint': 'https://contact-me.com'}}], 'version': 1, 'proof': {'type': 'SchnorrSecp256k12024', 'created': '2024-01-04T17:59:51Z', 'verificationMethod': '#initialKey', 'proofPurpose': 'capabilityInvocation', 'capability': 'urn:zcap:root:did%3Abtc1%3Ax1x03w9d88pwcj7cm4tjht5w3lg3rg2zjqzv072q2n2y8y2g9shqks5c7sl5', 'invocationTarget': 'did:btc1:x1x03w9d88pwcj7cm4tjht5w3lg3rg2zjqzv072q2n2y8y2g9shqks5c7sl5', 'capabilityAction': 'write', 'proofValue': 'z381yXYmxU8NudZ4HXY56DfMN6zfD8syvWcRXzT9xD9uYoQToo8QsXD7ahM3gXTzuay5WJbqTswt2BKaGWYn2hHhVFKJLXaDz'}}, 'proof': '8004953e000000000000007d94288c09736964656e6f646573945d948c176e6f6e5f6d656d626572736869705f6c65616664617461944e8c0c7369626c696e675f64617461944e752e'}}}

In [3]:
sidecar_history

{'initial_document': {'@context': ['https://www.w3.org/ns/did/v1'],
  'verificationMethod': [{'id': '#initialKey',
    'type': 'JsonWebKey',
    'publicKeyJwk': {'kty': 'EC',
     'crv': 'secp256k1',
     'x': '0233985507371e4a22cd2acc2cfed604efb2fc24400ff4a6bce8086561244f5a6d'},
    'controller': 'did:btc1:x1x03w9d88pwcj7cm4tjht5w3lg3rg2zjqzv072q2n2y8y2g9shqks5c7sl5'}],
  'authentication': ['#initialKey'],
  'assertionMethod': ['#initialKey'],
  'capabilityInvocation': ['#initialKey'],
  'capabilityDelegation': ['#initialKey'],
  'service': [{'id': '#initial_p2pkh',
    'type': 'SingletonBeacon',
    'serviceEndpoint': 'bitcoin:mvrk1NC2TWi3gnSUkD9NMyRFqtBGFdXhHv'},
   {'id': '#initial_p2wpkh',
    'type': 'SingletonBeacon',
    'serviceEndpoint': 'bitcoin:tb1q4prl9he7s99639g5kzp6pencna25ykmdd08h0k'},
   {'id': '#initial_p2tr',
    'type': 'SingletonBeacon',
    'serviceEndpoint': 'bitcoin:tb1p9s5fa89ypc6dnqmxkmclxa3ecfqvrfqchejq7d89agccm9c7u5fs86akaw'},
   {'id': '#smt_aggregated',
  

## R2. Verify Initial DID document against DID

As this is a sidecar DID, the did identifier string should be derived from the initial document

### R2.1 Split DID identifer into components

In [4]:
identifier_components = did_to_resolve.split(":")


### R2.2 Check validity of identifier

- The scheme MUST be the value did.
- The method MUST be the value btc. 

#### NOTE: IGNORING these for now:
- The version MUST be convertible to a positive integer value.
- SHOULD check if identifier is determinstic or sidecar

If any of these requirements fail, an invalidDid error MUST be raised. 

In [5]:
scheme = 'did'
method = 'btc1'

assert(scheme == identifier_components[0])
assert(method == identifier_components[1])

if (len(identifier_components) == 3):
    bech32_id = identifier_components[2]
elif (len(identifier_components) == 4):
    # TODO should cast identifier_components[2] to check that it is not an int and hence a version not network value
    network = identifier_components[2]
    bech32_id = identifier_components[3]
else:
    console.log("TODO: support optional version identifier components")

### R2.3 Verify identifier against intial DID document

- Check the ID in the initial DID document matches the DID being resolved.
- Remove ID from the initial DID document
- Hash and encodeBase64 the initial DID document check it matches the method identifier of the DID being resolved

In [6]:
from libbtc1.bech32 import encode_bech32_identifier, decode_bech32_identifier

type, hash_bytes = decode_bech32_identifier(bech32_id)


print("DID Type : ", type)

DID Type :  external


### Note: If initial document is NOT provided, the resolver MAY turn hash_bytes into a CID and query any Content Addressable Network they think appropriate.

Challenge is, there are potentially many configurations of CIDs that one can generate from a hash.

The other option is that the bech32_id encodes the CID itself.

In [7]:



initial_diddocument = sidecar_history["initial_document"]

assert(initial_diddocument["id"] == did_to_resolve)

In [8]:
import copy
partial_diddocument = copy.deepcopy(initial_diddocument)

del partial_diddocument["id"]
# TODO: May remove this pending issue in DID core
del partial_diddocument["verificationMethod"][0]["controller"]
partial_diddocument

{'@context': ['https://www.w3.org/ns/did/v1'],
 'verificationMethod': [{'id': '#initialKey',
   'type': 'JsonWebKey',
   'publicKeyJwk': {'kty': 'EC',
    'crv': 'secp256k1',
    'x': '0233985507371e4a22cd2acc2cfed604efb2fc24400ff4a6bce8086561244f5a6d'}}],
 'authentication': ['#initialKey'],
 'assertionMethod': ['#initialKey'],
 'capabilityInvocation': ['#initialKey'],
 'capabilityDelegation': ['#initialKey'],
 'service': [{'id': '#initial_p2pkh',
   'type': 'SingletonBeacon',
   'serviceEndpoint': 'bitcoin:mvrk1NC2TWi3gnSUkD9NMyRFqtBGFdXhHv'},
  {'id': '#initial_p2wpkh',
   'type': 'SingletonBeacon',
   'serviceEndpoint': 'bitcoin:tb1q4prl9he7s99639g5kzp6pencna25ykmdd08h0k'},
  {'id': '#initial_p2tr',
   'type': 'SingletonBeacon',
   'serviceEndpoint': 'bitcoin:tb1p9s5fa89ypc6dnqmxkmclxa3ecfqvrfqchejq7d89agccm9c7u5fs86akaw'},
  {'id': '#smt_aggregated',
   'type': 'SMTAggregatorBeacon',
   'serviceEndpoint': 'bitcoin:tb1pfdnyc8vxeca2zpsg365sn308dmrpka4e0n9c5axmp2nptdf7j6ts7eqhr8'}]}

In [9]:
from buidl.helper import sha256, str_to_bytes, bytes_to_str, encode_base58
import json

# Do we need canoncialization here?
data = str_to_bytes(json.dumps(partial_diddocument))

hashed_initial_doc = sha256(data)

assert(hash_bytes == hashed_initial_doc)

## R3. Initialize resolution variables

### R3.1 Initialize latest document to initial document

In [10]:
latest_document = copy.deepcopy(initial_diddocument)

### R3.2 Initialize activeBeaconIDs to empty array

The active beacon IDs will track the identifiers for beacon services in the latest DID document. This will change as DID updates are applied.

In [11]:
active_beacon_ids = []

### R3.3 Initialize current version to 0

All DID updates will have a version number, this specifies the order that the update should be applied starting from the initial document with a version of 0.

In [12]:
current_version = 0

### R3.4 Initialize beacon signals data to an empty array

Ordered signals data stores data about beacon signals retrieved from the bitcoin network

In [13]:
beacon_signals_data = []

### R3.5 Initialze processedUpdated and pendingUpdates to empty arrays.

- The processed updates array stores updates that have previously been applied to the DID document, this can be used to verify updates of the same sequence number are the same.
- The pending updates array stores updates that still need to be applied to the DID document. Likely becauase an update with an earlier sequence number has not yet been found.

In [14]:
processed_updates = []
pending_updates = []

## R4. Collect beacon signals (btc txs) from bitcoin for all beacons

Note: Is this all beacons in the initial DID document, or the current DID document. Or both? What about beacons that are added and removed in the inbetween states.

### R4.1 Get all beacon services from DID document

In [15]:
def find_beacon_services(service):
    # Note: also should support CID aggregated
    if service["type"] == "SingletonBeacon" or service["type"] == "SMTAggregatorBeacon":        
        return True
    else:
        return False

services = latest_document["service"]
beacon_services = list(filter(find_beacon_services, services))
beacon_services

[{'id': '#initial_p2pkh',
  'type': 'SingletonBeacon',
  'serviceEndpoint': 'bitcoin:mvrk1NC2TWi3gnSUkD9NMyRFqtBGFdXhHv'},
 {'id': '#initial_p2wpkh',
  'type': 'SingletonBeacon',
  'serviceEndpoint': 'bitcoin:tb1q4prl9he7s99639g5kzp6pencna25ykmdd08h0k'},
 {'id': '#initial_p2tr',
  'type': 'SingletonBeacon',
  'serviceEndpoint': 'bitcoin:tb1p9s5fa89ypc6dnqmxkmclxa3ecfqvrfqchejq7d89agccm9c7u5fs86akaw'},
 {'id': '#smt_aggregated',
  'type': 'SMTAggregatorBeacon',
  'serviceEndpoint': 'bitcoin:tb1pfdnyc8vxeca2zpsg365sn308dmrpka4e0n9c5axmp2nptdf7j6ts7eqhr8'}]

### R4.2 Fetch beacon signals for each beacon service

In [16]:
import requests

## Define function
def fetch_beacon_signals(network, beacon_service):
    # 1. initialize beacon signals array
    beacon_signals = []
    # 2. Set `beaconAddressURI` to `beaconService.serviceEndpoint`
    beacon_address_uri = beacon_service["serviceEndpoint"]
    # 3. Set `beaconAddress` to the conversion of `beaconAddressURI` to a bitcoin address following BIP21.
    beacon_address = beacon_address_uri.split(':')[1]
    print("\nBeacon address", beacon_address)
    # 4. Set `spendTxs` to the spends on the Bitcoin blockchain `network` from the `beaconAddress`. 
    response = requests.get(f'https://mempool.space/{network}/api/address/{beacon_address}/txs')
    # Tx data
    txs_data = response.json()
    # TODO: call tx data.
    spend_txs_data = []
    

    for tx_data in txs_data:
        # Only care about bitcoin transactions that have been accepted into the chain.
        # Should have X number of confirmations
        # Check if the address is an input (it's spending an UTXO)
        if any(vin['prevout']['scriptpubkey_address'] == beacon_address for vin in tx_data['vin']):
            spend_txs_data.append(tx_data)

    if len(spend_txs_data) == 0:
        print(f"No beacon signals found from {beacon_service["type"]} with address {beacon_address}")

    # Now, spend_txs contains all transactions where the address spent an UTXO
    for tx_data in spend_txs_data:
        print(tx_data)

        if (tx_data["status"]["confirmed"]):
            signal = {
                "beacon_id": beacon_service["id"],
                "beacon_type": beacon_service["type"],
                "block_height": tx_data["status"]["block_height"],
                "tx": tx_data
            }
            beacon_signals_data.append(signal)
        else:
            print("Signal not yet confirmed in bitcoin blockchain")

    return beacon_signals_data

In [17]:

## Run fetch_beacon_signals for each beacon service not currently fetched from
for beacon_service in beacon_services:
    if (beacon_service["id"] not in active_beacon_ids):
        # Fetch beacon signals
        print("\nFetch signals")
        network = "signet"
        beacon_signals = fetch_beacon_signals(network, beacon_service)

        beacon_signals_data += beacon_signals
        ## Signals from beacon have been processed
        active_beacon_ids.append(beacon_service["id"])


Fetch signals

Beacon address mvrk1NC2TWi3gnSUkD9NMyRFqtBGFdXhHv
No beacon signals found from SingletonBeacon with address mvrk1NC2TWi3gnSUkD9NMyRFqtBGFdXhHv

Fetch signals

Beacon address tb1q4prl9he7s99639g5kzp6pencna25ykmdd08h0k
No beacon signals found from SingletonBeacon with address tb1q4prl9he7s99639g5kzp6pencna25ykmdd08h0k

Fetch signals

Beacon address tb1p9s5fa89ypc6dnqmxkmclxa3ecfqvrfqchejq7d89agccm9c7u5fs86akaw
No beacon signals found from SingletonBeacon with address tb1p9s5fa89ypc6dnqmxkmclxa3ecfqvrfqchejq7d89agccm9c7u5fs86akaw

Fetch signals

Beacon address tb1pfdnyc8vxeca2zpsg365sn308dmrpka4e0n9c5axmp2nptdf7j6ts7eqhr8
{'txid': 'bb1127988db99a6fa710cec1f7514de8995b69ccc6bfeb89a6877a5424ce69c8', 'version': 1, 'locktime': 0, 'vin': [{'txid': 'e067ea53780be9ca587bdc76f0bf81e949bd3e2e9052d99049eaac5feffb7c58', 'vout': 1, 'prevout': {'scriptpubkey': '51204b664c1d86ce3aa106088ea909c5e76ec61b76b97ccb8a74db0aa615b53e9697', 'scriptpubkey_asm': 'OP_PUSHNUM_1 OP_PUSHBYTES_32 4b664

## R4.3 Check processed all beacons in services. 

This is a proposed algorithm. It is left as a TODO:

If `activeBeaconIDs.length !== beaconServices.length`:
  - Identify beaconIds in `activeBeaconIDs` and NOT in the `beaconServices` array. Store in `removedBeaconIDs`.
  - Remove all signals from the `orderedSignals` array with a `signal.beaconId` value in the `removedBeaconIDs` array.
  - Remove `removedBeaconIDs` from the `activeBeaconIDs` array.
  - Set `removedBeaconIDs` to an empty array
  - Expect the length of `activeBeaconIDs` to equal the length of `beaconServices`

**Question:** If a beacon is removed from a DID document in a valid update, do we remove any beacon signals from that beacon that were broadcast to the network at a blocktime that is greater than the beacon signal that contained the DID update that removed the beacon?if (len(active_beacon_ids) != len(beacon_services)):
    print("TODO")

In [18]:
if (len(active_beacon_ids) != len(beacon_services)):
    print("TODO")
else:
    print("All beacon signals returned from active beacon services in DID document")

All beacon signals returned from active beacon services in DID document


### R4.4 If no beacon signals AND no pending updates to process, return latest DID document



In [19]:
if (len(beacon_signals_data) == 0):
    if (len(pending_updates) == 0):    
        print("RESOLUTION COMPLETE\n\n")
        print(f"No more signals to process, returning latest document for {did_to_resolve}\n")
        print(latest_document)
    else:
        raise "Error: no more beacon signals but still have pending updates to process. Unable to build full history of DID document updates from available beacon signals"
else:
    print("Still have signals to process")

Still have signals to process


### R4.5 Sort beacon signals data using block height

In [20]:
beacon_signals_data.sort(key=lambda x: x["block_height"])

## R5. Process Beacon Signals to retrieve DID updates according to beacon type

This notebook is only processing beacon signals of the SMTAggregatedBTCBeacon type.

For each smt beacon signal we:

1. Get the txid
2. Get the sidecar data for that txid (If no sidecar data, throw error)
3. Get SMT root from the first tx output of the signal
4. Check the sidecar provided merkle proof and did update invocation (may be null) using the DID being resolved as the key against the SMT root.
5. If verified, add did update invocation to list of pending updates



In [21]:
from buidl.script import Script
from buidl.helper import bytes_to_str
from smt.tree import SparseMerkleTree
from smt.proof import verify_proof, SparseMerkleProof
from smt.utils import DEFAULTVALUE
import pickle



def process_smt_beacon_signal(tx):
    # 1. Get txid
    txid = tx["txid"]
    # 2. Get sidecar data for txid
    signal_sidecar_data = sidecar_history["signals"].get(txid)
    if signal_sidecar_data == None:
        # Throw error if no sidecar data found
        raise Exception("No signal data provided for signal with txid : " + txid)

    # 3. Get SMT root from the first tx output of the signal bitcoin tx
    # Fetch txout from signal
    tx_out = tx["vout"][0]

    # Check txOut is of the format [OP_RETURN, 32BYTE_CID], if not MUST raise invalidBeaconSignal error.
    assert(tx_out["scriptpubkey_type"] == "op_return")
    scriptpubkey_hex = tx_out["scriptpubkey"]
    script = Script.parse_hex(scriptpubkey_hex)
    
    assert(len(script.commands[1]) == 32)
    # Fetch SMT root from txout script pubkey
    smt_root = script.commands[1]
    print(f"SMT root: {smt_root}")

    # 4. Check the sidecar provided merkle proof and did update invocation (may be null) 
    # using the DID being resolved as the key against the SMT root.
    
    # Fetch SMT proof from sidecar data
    proof_hex = signal_sidecar_data["proof"]
    proof = bytes.fromhex(proof_hex)
    proof_data = pickle.loads(proof)
    smt_proof = SparseMerkleProof(sidenodes=proof_data['sidenodes'], non_membership_leafdata=proof_data["non_membership_leafdata"], siblingdata=proof_data["sibling_data"])
    

     # Key to SMT leaf
    did_bytes = str_to_bytes(did_to_resolve)
    # Data at leaf. Either None or an invocation
    did_update_invocation = signal_sidecar_data["update"]

    leaf_value = DEFAULTVALUE
    if did_update_invocation:
        leaf_value = str_to_bytes(json.dumps(did_update_invocation))

    verified = verify_proof(smt_proof, smt_root, did_bytes, leaf_value)
    print(f"\nSMT proof for {did_to_resolve} and Update \n{did_update_invocation}\nVerified : {verified}\n")

    # 5. If verified, add did update invocation to list of pending updates
    if verified:
        print(f"\nverified update for {did_to_resolve}:\n{did_update_invocation}\n")
        if did_update_invocation:
            pending_updates.append(did_update_invocation)

In [27]:


# Run process smt beacon signals for all SMTAggregatedBTCBeacons signals in the beacon_signals_data
# TODO: remove signals from ordered signals data as you process them.
for signal_data in beacon_signals_data:
    beacon_type = signal_data["beacon_type"]
    if beacon_type == "SMTAggregatorBeacon":
            print(f"Processing beacon signal of Type {beacon_type}")
            signal_tx = signal_data["tx"]
            process_smt_beacon_signal(signal_tx)
            beacon_signals_data.remove(signal_data)
    else:
        print("Not Implemented: Unable to process beacon of type " + beacon_type)
        break
    
    

Processing beacon signal of Type SMTAggregatorBeacon


Exception: No signal data provided for signal with txid : e067ea53780be9ca587bdc76f0bf81e949bd3e2e9052d99049eaac5feffb7c58

In [23]:
beacon_signals_data

[{'beacon_id': '#smt_aggregated',
  'beacon_type': 'SMTAggregatorBeacon',
  'block_height': 185525,
  'tx': {'txid': 'e067ea53780be9ca587bdc76f0bf81e949bd3e2e9052d99049eaac5feffb7c58',
   'version': 1,
   'locktime': 0,
   'vin': [{'txid': '222b2223defe2b5cdb6261763753b8c755ce73a77d1bec596a6b308d8c60a2d4',
     'vout': 1,
     'prevout': {'scriptpubkey': '51204b664c1d86ce3aa106088ea909c5e76ec61b76b97ccb8a74db0aa615b53e9697',
      'scriptpubkey_asm': 'OP_PUSHNUM_1 OP_PUSHBYTES_32 4b664c1d86ce3aa106088ea909c5e76ec61b76b97ccb8a74db0aa615b53e9697',
      'scriptpubkey_type': 'v1_p2tr',
      'scriptpubkey_address': 'tb1pfdnyc8vxeca2zpsg365sn308dmrpka4e0n9c5axmp2nptdf7j6ts7eqhr8',
      'value': 100000},
     'scriptsig': '',
     'scriptsig_asm': '',
     'witness': ['7ee0e03101593578f98f597b8052efb9b91680708da66f8230148393b2bc6d700d3d271dae5aae54301748e86e54e1e4904b623798639803b6b7ead76049b83a'],
     'is_coinbase': False,
     'sequence': 4294967295}],
   'vout': [{'scriptpubkey': '6a20

## R6 Apply pending upates to DID document

### R6.1 If no pending updates, return to step R4 to check for additional beacon signals

Applying updates to a DID document may have added additional beacons from which to fetch beacon signals to process

In [24]:
if (len(pending_updates) == 0):
    print("No pending updates to process, Return to step R4")

No pending updates to process, Return to step R4


### R6.2 Order pending updates by version



In [25]:
pending_updates.sort(key=lambda x: x["version"])

### R6.3 Get pending update from start of list

In [26]:
next_update = pending_updates[0]

IndexError: list index out of range

### R6.4 Process next update

- If `next_update.version` <= `current_version`:
    - Find `process_update_invocation` of that order
    - Check hash of `process_update_invocation` is equal to `next_update`. If not, then MUST raise a `latePublish` error.
    - Remove update from `pending_updates`

- Else if `next_update.version` == `current_version + 1`:
    - Apply `next_update.updateTransformation` deltas to the `latestDocument`
    - Set `current_version` to `next_update.version`
    - Push `next_update` to `processed_updates`
    - Remove update from `pending_updates`
- Else:
    - Return to step 4 to attempt to discover more updates. Need to find next one in the version


#### R6.4.1 Update sequence number already seen. Check update is identitcal to previously processed with same sequence number

In [29]:
version = next_update["version"]

In [30]:
processed_updates

[]

In [31]:
if version <= current_version:
    print("Version already seen, checking for equivalence")
    # Find previous update with same version
    prev_processed_updates = [p for p in processed_updates if p["version"] == version]
    # pending_updates.remove(next_update)
    if (len(prev_processed_updates) != 0):
        prev_bytes = str_to_bytes(json.dumps(prev_processed_updates[0]))
        hash_prev = sha256(prev_bytes)
        new_bytes = str_to_bytes(json.dumps(next_update))
        hash_new = sha256(new_bytes)
        if (hash_prev != hash_new):
            raise Exception("Late Publish Error")
        else:
            print("Hashes are equal, update has already been processed.")
            print("Process next update: Return to Step R6.1")
    else:
        raise Exception("Previously processed update not found")

        

#### R6.4.2 Update is next in sequence. Process update

The major TODO here is find/develop a library for applying arbitrary JSON0 transformation. See https://github.com/ottypes/json0

In [32]:
if version == current_version + 1:
    # Apply didUpdateInvocation.update deltas to the latestDocument
    update_transformation = next_update["updateTransformation"]
    print("Applying update to latest DID document :", update_transformation)
    
    # TODO: find/implement a OT python library. 
    # This is just hardcoded for the moment.
    latest_document["service"].append(update_transformation['li'])

    # Add update to list of processed updates
    processed_updates.append(next_update)
    # Bump version
    current_version += 1
    # Remove update from pending updates
    pending_updates.remove(next_update)
    print("\n\nUpdate processed. Process next update. Return to Step R6.1")

Applying update to latest DID document : {'p': ['service', 4], 'li': {'id': '#linked-domain', 'type': 'LinkedDomains', 'serviceEndpoint': 'https://contact-me.com'}}


Update processed. Process next update. Return to Step R6.1


### R6.4.3 Update is higher than next in sequence. Leave in pending updates

Unable to process any more updates, need to find the next update in the sequence. Return to step R4 to look for additional beacon signals

In [33]:
if version >= current_version + 1:
    print("Unable to process any more updates, need to find the next update in the sequence. Return to step R4 to look for additional beacon signals")

## Resolution of a DID is complete once there are no more beacon signals to process AND all updates from beacon signals have been applied in sequential order.

Resolution is complete in step R4.4.